# Read directly from IIASA data resources

IIASA's new [scenario explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer/#/workspaces) is not only a great resource on its own, but it also allows the underlying datasets to be directly queried.
**pyam** takes advantage of this ability to allow you to easily pull data and work with it.

In [ ]:
import pyam

Accessing an explorer is done via a `Connection` object.
By default, your can connect to all public scenario explorers instances. 

In [ ]:
conn = pyam.iiasa.Connection()
conn.valid_connections

If you have additional credentials, you can supply them as well via the `creds` key-word argument:

```
pyam.iiasa.Connection(creds=(<username>, <password>))
```

In this example, we will be pulling data from the Special Report on 1.5C explorer. This can be done either via the constructor:

```
pyam.iiasa.Connection('IXSE_SR15')
```

or, if you want to query multiple databases, via the explicit `connect()` method:

```
conn = pyam.iiasa.Connection()
conn.connect('IXSE_SR15')
```

We also provide some convenience functions to shorten the amount of code you have to write. Under the hood, `read_iiasa()` is just opening an connection to a database and making a query on that data.
In this tutorial, we will query specific subsets of data in a manner similar to `pyam.IamDataFrame.filter()`.

In [ ]:
df = pyam.read_iiasa(
    'iamc15',
    model='MESSAGEix*', 
    variable=['Emissions|CO2', 'Primary Energy|Coal'], 
    region='World',
    meta=['category']
)

Here we pulled out all times series data for model(s) that start with 'MESSAGEix' that are in the 'World' region and associated with the two named variables. We also added the "category" metadata, which tells us the climate impact categorisation of each scenario as assessed in the IPCC SR15.

Let's plot CO2 emissions.

In [ ]:
ax = df.filter(variable='Emissions|CO2').line_plot(
    color='category', 
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)

And now continue doing all of your analysis!

In [ ]:
ax = df.scatter(
    x='Primary Energy|Coal', 
    y='Emissions|CO2', 
    color='category',
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)

## Exploring the Data Source

If you're interested in what data is actually in the data source, you can use `pyam.iiasa.Connection` to do so.

In [ ]:
conn = pyam.iiasa.Connection('iamc15')

The `conn` object has a number of useful functions for listing what's in the dataset. A few of them are shown below.

In [ ]:
conn.models().head()

In [ ]:
conn.scenarios().head()

In [ ]:
conn.variables().head()

In [ ]:
conn.regions().head()

A number of different kinds of indicators are available for model/scenario combinations.
We queried the "category" metadata in the above example, but there are many more. You can see them with

In [ ]:
conn.available_metadata().head()

You can directly query the `Connection`, which will give you a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [ ]:
df = conn.query(
    model='MESSAGEix*', 
    variable=['Emissions|CO2', 'Primary Energy|Coal'], 
    region='World'
)
df.head()

And you can easily turn this into a `pyam.IamDataFrame` to continue your analysis.

In [ ]:
df = pyam.IamDataFrame(df)
ax = df.filter(variable='Primary Energy|Coal').line_plot(
    color='scenario', 
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)